In [2]:
import sys
import os

wd = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(wd,'bin'))

import libsbml
import importlib
import amici
import numpy as np
import re
import pandas as pd
from antimony import *
from modules.copyDir import copyDirectory

# Optional packages to import
import amici.plotting
import matplotlib.pyplot as plt
from time import time
import matplotlib as mpl
from modules.RunSPARCED import RunSPARCED

mpl.rcParams['figure.dpi'] = 300



In [3]:
sbml_file = "SPARCED.xml"
model_name= sbml_file[0:-4]
model_output_dir = model_name
sys.path.insert(0, os.path.join(wd,model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()

species_all = list(model.getStateIds())


Vn = 1.75E-12
Vc = 5.25E-12

solver = model.getSolver()          # Create solver instance
solver.setMaxSteps = 1e10

ts = 30

model.setTimepoints(np.linspace(0,ts))

In [4]:
STIMligs = [1000.0,100.0,100.0,100.0,100.0,100.0,1721.0] # EGF, Her, HGF, PDGF, FGF, IGF, INS

STIMligs_id = ['E', 'H', 'HGF', 'P', 'F', 'I', 'INS']



spIn2 = np.array(model.getInitialStates())

for s,sp in enumerate(STIMligs_id):
    spIn2[species_all.index(sp)] = STIMligs[s]


model.setInitialStates(spIn2)

flagD = 1

th = 72

xoutS_all, xoutG_all, tout_all = RunSPARCED(flagD,th,spIn2,[],sbml_file,model)

AttributeError: 'NoneType' object has no attribute 'getCompartment'